In [25]:
# noexport

import os
os.system('export_notebook identify_domain.ipynb')

0

In [1]:
from tmilib import *

In [3]:
#user = get_test_users()[0]
#print user

In [4]:
#tab_focus_times = get_tab_focus_times_for_user(user)
#print tab_focus_times[0].keys()

In [5]:
#print active_second_to_domain_id.keys()[:10]

In [2]:
def get_recently_seen_domain_stats_for_user(user):
  ordered_visits = get_history_ordered_visits_corrected_for_user(user)
  ordered_visits = exclude_bad_visits(ordered_visits)
  active_domain_at_time = get_active_domain_at_time_for_user(user)
  active_seconds_set = set(get_active_seconds_for_user(user))
  active_second_to_domain_id = get_active_second_to_domain_id_for_user(user)
  recently_seen_domain_ids = [-1]*100
  stats = Counter()
  nomatch_domains = Counter()
  for idx,visit in enumerate(ordered_visits):
    if idx+1 >= len(ordered_visits):
      break
    next_visit = ordered_visits[idx+1]
    cur_domain = url_to_domain(visit['url'])
    cur_domain_id = domain_to_id(cur_domain)
    if cur_domain_id != recently_seen_domain_ids[-1]:
      recently_seen_domain_ids.append(cur_domain_id)
    next_domain = url_to_domain(next_visit['url'])
    next_domain_id = domain_to_id(next_domain)
    cur_time_sec = int(round(visit['visitTime'] / 1000.0))
    next_time_sec = int(round(visit['visitTime'] / 1000.0))
    for time_sec in xrange(cur_time_sec, next_time_sec+1):
      if time_sec not in active_seconds_set:
        continue
      ref_domain_id = active_second_to_domain_id[str(time_sec)]
      stats['total'] += 1
      if cur_domain_id == ref_domain_id:
        if next_domain_id == cur_domain_id:
          stats['first and next equal and correct'] += 1
          continue
        else:
          stats['first correct only'] += 1
          continue
      else:
        if next_domain_id == cur_domain_id:
          stats['both incorrect'] += 1
          found_match = False
          for i in range(1,101):
            if recently_seen_domain_ids[-1-i] == ref_domain_id:
              stats['nth previous correct ' + str(abs(i))] += 1
              stats['some previous among past 100 correct'] += 1
              found_match = True
              break
          if not found_match:
            ref_domain = id_to_domain(ref_domain_id)
            if ref_domain == 
            stats['no match found'] += 1
            nomatch_domains[id_to_domain(ref_domain_id)] += 1
          continue
        if next_domain_id == ref_domain_id:
          stats['next correct only'] += 1
          continue
  return {
    'stats': stats,
    'nomatch_domains': nomatch_domains,
  }

In [3]:
total_nomatch_domains = Counter()

for user in get_test_users():
  for k,v in get_recently_seen_domain_stats_for_user(user)['nomatch_domains'].viewitems():
    total_nomatch_domains[k] += v


In [4]:
#print total_nomatch_domains
print_counter(total_nomatch_domains)

newtab 6069
klbibkeccnjlkjkiokjodocebajanakg 582
bookmarks 246
www.google.com 199
 178
hitgrabber.net 54
history 41
goo.gl 41
local-ntp 41
gmail.com 35
www.ow.ly 34
uts.saintleo.edu 28
maps.google.com 24
www.gmail.com 23
www.fhb.com 21
www.hotmail.com 20
docs.google.com 20
yahoo.com 19
bit.ly 19
tmi.stanford.edu 18
www.facebook.com 17
elliman.com 16
mail.yahoo.com 16
click.messages.webmd.com 15
www.google.co.in 15
click.harrispollonline.com 15
www.techpowerup.com 14
apps 14
www.amazon.com 13
www.bit.ly 12
www.bigcattracks.com 12
extensions 12
amazon.com 11
www.instructables.com 11
youtu.be 10
yhoo.it 10
www.pandora.com 10
www.google.com.tr 10
mturk.com 10
www.reddit.com 10
movies.netflix.com 10
www.financial-net.com 9
tinyurl.com 9
click.email.time.com 9
www.turkitron.com 9
paypal.com 8
www.stream2watch.co 8
adclick.g.doubleclick.net 8
ws.surveymyopinion.com 7
instagram.com 7
lifeleadership.com 7
erlibird.com 6
link.digg.com 6
www.surveymyopinion.com 6
gvsupsychology.qualtrics.com 6
fa

In [16]:
#total_stats = Counter({'total': 544544, 'first and next equal and correct': 351081, 'first correct only': 88522, 'both incorrect': 51663, 'some previous among past 20 correct': 41231, 'nth previous correct 1': 31202, 'next correct only': 23569, 'no match found': 10432, 'nth previous correct 2': 3311, 'nth previous correct 3': 1635, 'nth previous correct 4': 905, 'nth previous correct 5': 862, 'nth previous correct 6': 545, 'nth previous correct 7': 412, 'nth previous correct 8': 357, 'nth previous correct 9': 269, 'nth previous correct 10': 259, 'nth previous correct 13': 234, 'nth previous correct 11': 229, 'nth previous correct 12': 190, 'nth previous correct 15': 183, 'nth previous correct 14': 140, 'nth previous correct 17': 139, 'nth previous correct 20': 95, 'nth previous correct 16': 90, 'nth previous correct 19': 88, 'nth previous correct 18': 86})

total_stats = Counter()

for user in get_test_users():
  for k,v in get_recently_seen_domain_stats_for_user(user).viewitems():
    total_stats[k] += v

#total_stats = Counter({'total': 544544, 'first and next equal and correct': 351081, 'first correct only': 88522, 'both incorrect': 51663, 'some previous among past 20 correct': 41136, 'nth previous correct 2': 31202, 'next correct only': 23569, 'no match found': 10527, 'nth previous correct 3': 3311, 'nth previous correct 4': 1635, 'nth previous correct 5': 905, 'nth previous correct 6': 862, 'nth previous correct 7': 545, 'nth previous correct 8': 412, 'nth previous correct 9': 357, 'nth previous correct 10': 269, 'nth previous correct 11': 259, 'nth previous correct 14': 234, 'nth previous correct 12': 229, 'nth previous correct 13': 190, 'nth previous correct 16': 183, 'nth previous correct 15': 140, 'nth previous correct 18': 139, 'nth previous correct 17': 90, 'nth previous correct 20': 88, 'nth previous correct 19': 86})

In [17]:
print total_stats

Counter({'total': 544544, 'first and next equal and correct': 351081, 'first correct only': 88522, 'both incorrect': 51663, 'some previous among past 20 correct': 41231, 'nth previous correct 1': 31202, 'next correct only': 23569, 'no match found': 10432, 'nth previous correct 2': 3311, 'nth previous correct 3': 1635, 'nth previous correct 4': 905, 'nth previous correct 5': 862, 'nth previous correct 6': 545, 'nth previous correct 7': 412, 'nth previous correct 8': 357, 'nth previous correct 9': 269, 'nth previous correct 10': 259, 'nth previous correct 13': 234, 'nth previous correct 11': 229, 'nth previous correct 12': 190, 'nth previous correct 15': 183, 'nth previous correct 14': 140, 'nth previous correct 17': 139, 'nth previous correct 20': 95, 'nth previous correct 16': 90, 'nth previous correct 19': 88, 'nth previous correct 18': 86})


In [18]:
def sumkeys(d, *args):
  return sum(d.get(x, 0.0) for x in args)

In [21]:
norm = {k:float(v)/total_stats['total'] for k,v in total_stats.viewitems()}
print 'select prev gets answer correct', sumkeys(norm, 'first and next equal and correct', 'first correct only')
print 'prev or next gets answer correct', sumkeys(norm, 'first and next equal and correct', 'first correct only', 'next correct only')
for i in range(1, 101):
  sumprev = sum([norm.get('nth previous correct '+str(x),0.0) for x in range(i+1)])
  print 'prev or next or past ' + str(i), sumkeys(norm, 'first and next equal and correct', 'first correct only', 'next correct only')+sumprev

select prev gets answer correct 0.807286463536
prev or next gets answer correct 0.850568549098
prev or next or past 1 0.907867867427
prev or next or past 2 0.913948184169
prev or next or past 3 0.916950696362
prev or next or past 4 0.918612637363
prev or next or past 5 0.92019561321
prev or next or past 6 0.921196450608
prev or next or past 7 0.921953046953
prev or next or past 8 0.922608641359
prev or next or past 9 0.923102632661
prev or next or past 10 0.923578259975
prev or next or past 11 0.923998795322
prev or next or past 12 0.924347711112
prev or next or past 13 0.924777428454
prev or next or past 14 0.925034524299
prev or next or past 15 0.925370585297
prev or next or past 16 0.925535861198
prev or next or past 17 0.925791120644
prev or next or past 18 0.925949050949
prev or next or past 19 0.926110654052
prev or next or past 20 0.926285111947
